In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import datetime as dt
import copy
import matplotlib.pyplot as plt
from yahooquery import Ticker

In [2]:
#importando base
df_ibrx = pd.read_excel(r'C:\Users\drsan\OneDrive\Área de Trabalho\Python\ETFs_Onshore.xlsx',sheet_name = 'IBRX100')
df_ibrx.head()

,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Setor,Subsetor,Segmento
0,ABEV3,AMBEV S/A,ON,4.354.228.928,2936.0,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
1,ALPA4,ALPARGATAS,PN N1,187.007.704,328.0,Consumo Cíclico,"Tecidos, Vestuário e Calçados",Calçados
2,ALSO3,ALIANSCSONAE,ON NM,127.374.163,186.0,Financeiro,Exploração de Imóveis,Exploração de Imóveis
3,AZUL4,AZUL,PN N2,317.471.474,363.0,Bens Industriais,Transporte,Transporte Aéreo
4,B3SA3,B3,ON NM,2.046.021.644,6301.0,Financeiro,Serviços Financeiros Diversos,Serviços Financeiros Diversos


In [3]:
# dando aquele tapa
df = df_ibrx
df = pd.DataFrame(df) 
df = df.dropna()
df['Part. (%)'] /= 100000
df.rename(columns={'Código\xa0':'Codigo'}, inplace = True)
df['Codigo'] = df['Codigo'] + ".SA"
tickers = df['Codigo']

In [ ]:
start = dt.date.today() + dt.timedelta(1) - dt.timedelta(59)
end = dt.date.today() + dt.timedelta(1)
prices = {}
for t in tickers:
    try:
        prices[t] = yf.download(t,start,end,interval='1d')  
    except: 
        pass
    prices[t].dropna(how='all', inplace = True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
prices2 = copy.deepcopy(prices)
for t in tickers:
    prices2[t]['roll_avg_5d'] = prices2[t]['Close'].rolling(5).mean()
    prices2[t]['roll_avg_20d'] = prices2[t]['Close'].rolling(20).mean()
    prices2[t]['roll_max_5d'] = prices2[t]['Close'].rolling(5).max()
    prices2[t]['roll_max_20d'] = prices2[t]['Close'].rolling(20).max()
    prices2[t]['roll_min_5d'] = prices2[t]['Close'].rolling(5).min()
    prices2[t]['roll_min_20d'] = prices2[t]['Close'].rolling(20).min()
    prices2[t].dropna(inplace=True)

In [ ]:
prices3 = pd.DataFrame(columns=['ticker','Setor', 'Subsetor', 'Segmento', 'Close', 'Close_D-1', 'roll_avg_5d', 'roll_avg_20d',
                                'roll_max_5d', 'roll_max_20d', 'roll_min_5d', 'roll_min_20d'])
for t in range(len(tickers)):
    prices3.loc[t,'ticker'] = str(tickers[t]).replace('.SA','')
    prices3.iloc[t,1] = df[df['Codigo'] == tickers[t]]['Setor'].to_list()[0]
    prices3.iloc[t,2] = df[df['Codigo'] == tickers[t]]['Subsetor'].to_list()[0]
    prices3.iloc[t,3] = df[df['Codigo'] == tickers[t]]['Segmento'].to_list()[0]
    prices3.iloc[t,4] = prices2[tickers[t]]['Close'][-1]
    prices3.iloc[t,5] = prices2[tickers[t]]['Close'][-2]
    prices3.iloc[t,6] = prices2[tickers[t]]['roll_avg_5d'][-1]
    prices3.iloc[t,7] = prices2[tickers[t]]['roll_avg_20d'][-1]
    prices3.iloc[t,8] = prices2[tickers[t]]['roll_max_5d'][-1]
    prices3.iloc[t,9] = prices2[tickers[t]]['roll_max_20d'][-1]
    prices3.iloc[t,10] = prices2[tickers[t]]['roll_min_5d'][-1]
    prices3.iloc[t,11] = prices2[tickers[t]]['roll_min_20d'][-1]   
    

In [ ]:
prices3